In [ ]:
def get_attributes(object):
    attributes = dir(object)
    filtered_attributes = [
        attr for attr in attributes if not attr.startswith('__')]
    return filtered_attributes


def get_methods(object):
    methods = dir(object)
    filtered_methods = [
        method for method in methods if not method.startswith('__') and callable(getattr(object, method))]
    return filtered_methods


def get_properties(object):
    properties = dir(object)
    filtered_properties = [
        prop for prop in properties if not prop.startswith('__') and not callable(getattr(object, prop))]
    return filtered_properties


In [ ]:
import requests
import json


In [ ]:
base_url = 'https://openalex.org/'
